# Procesamiento de datos - ETL

# Extract

In [1]:
import numpy as np
import pandas as pd
import requests
import os
from zipfile import ZipFile
from sqlalchemy import create_engine

Se extrae la data de la encuesta de 2021 del sitio [oficial de Stack Overflow](https://insights.stackoverflow.com/survey) con web scraping.

In [2]:
# Get the 2021 survey (in a zip file) from Stack Overflow
path = 'https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2021.zip'
response = requests.get(path)
print(response.status_code)

# Save the file locally
local_path = os.path.join(os.getcwd(), os.pardir, 'data', 'raw', 'survey_2021.zip')
with open(local_path, "wb") as f:
    f.write(response.content)

200


In [3]:
# Get the list of files
path_save = os.path.join(os.getcwd(), os.pardir, 'data', 'raw')

with ZipFile(local_path, "r") as f:
    file_names = f.namelist()
    print(file_names)
    csv_file_path_1 = f.extract(file_names[2], path_save)
    print(csv_file_path_1)
    csv_file_path_2 = f.extract(file_names[3], path_save)
    print(csv_file_path_2)

['README_2021.txt', 'so_survey_2021.pdf', 'survey_results_public.csv', 'survey_results_schema.csv']
/work/survey-dev/data/raw/survey_results_public.csv
/work/survey-dev/data/raw/survey_results_schema.csv


# Transform

In [4]:
# Read de data
path = os.path.join(os.getcwd(), os.pardir, 'data', 'raw')
df_schema = pd.read_csv(os.path.join(path, 'survey_results_schema.csv'))
df_survey = pd.read_csv(os.path.join(path, 'survey_results_public.csv'))

### Filtro de datos

El `df_schema` contiene las preguntas usadas en la encuesta.

In [5]:
# Explore what data to use
cols = ['qname', 'question']
df_schema[cols]

,qname,question
0,S0,"<div><span style=""font-size:19px;""><strong>Hel..."
1,MetaInfo,Browser Meta Info
2,S1,"<span style=""font-size:22px; font-family: aria..."
3,MainBranch,Which of the following options best describes ...
4,Employment,Which of the following best describes your cur...
5,Country,"Where do you live? <span style=""font-weight: b..."
6,US_State,<p>In which state or territory of the USA do y...
7,UK_Country,In which part of the United Kingdom do you liv...
8,S2,"<span style=""font-size:22px; font-family: aria..."
9,EdLevel,Which of the following best describes the high...


In [6]:
df_survey.sample(5)

,ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,...,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
21775,21776,I am a developer by profession,Employed full-time,United States of America,Colorado,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",18 - 24 years,"Other online resources (ex: videos, blogs, etc...",7,...,25-34 years old,Man,No,NaN,Prefer not to say,None of the above,Prefer not to say,Appropriate in length,Easy,NaN
79414,79415,I am a student who is learning to code,"Student, full-time",India,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",7,...,18-24 years old,Man,No,Straight / Heterosexual,Southeast Asian;East Asian,I am blind / have difficulty seeing,Prefer not to say,Too long,Neither easy nor difficult,NaN
197,198,I am a student who is learning to code,"Independent contractor, freelancer, or self-em...",Nepal,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",18 - 24 years,Coding Bootcamp;Other online resources (ex: vi...,3,...,18-24 years old,Man,No,Straight / Heterosexual,South Asian,None of the above,None of the above,Appropriate in length,Easy,NaN
20628,20629,I code primarily as a hobby,"Student, full-time",Canada,NaN,NaN,Primary/elementary school,5 - 10 years,"Other online resources (ex: videos, blogs, etc...",8,...,Under 18 years old,Man,No,Prefer not to say,Prefer not to say,None of the above,Prefer not to say,Appropriate in length,Easy,NaN
17453,17454,I am a developer by profession,Employed part-time,United States of America,Florida,NaN,Some college/university study without earning ...,11 - 17 years,"Other online resources (ex: videos, blogs, etc...",8,...,18-24 years old,Man,No,Straight / Heterosexual,White or of European descent;Hispanic or Latin...,None of the above,I have a mood or emotional disorder (e.g. depr...,Appropriate in length,Easy,8000.0


Seleccionar las columnas que pueden ayudar a contestar las preguntas (mostradas en el tercer notebook). Y también los países hispanohablantes para filtrar la data (también se incluyó a Brasil).

In [7]:
# Select columns
columns = ['ResponseId', 'Age', 'Gender', 'Sexuality', 'Country',
            'EdLevel', 'LearnCode', 'YearsCode', 'YearsCodePro',
            'Employment', 'ConvertedCompYearly']

# Filter registers by countries
latam = ['Peru', 'Colombia', 'Chile', 'Argentina', 'Costa Rica', 'Bolivia',
            'Uruguay', 'Mexico', 'Venezuela, Bolivarian Republic of...'
            'Dominican Republic', 'Ecuador', 'Guatemala', 'Paraguay', 'Panama',
            'El Salvador', 'Nicaragua',  'Brazil', 'Spain']

# New dataset
in_latam = df_survey.Country.isin(latam)
df = df_survey[in_latam][columns].set_index('ResponseId')

In [8]:
s = df.shape
p = s[0] / df_survey.shape[0] * 100
print(f'Registros: {s[0]}')
print(f'Porcentaje total del dataset: {round(p,2)}%')
print(f'Columnas: {s[1]}')

Registros: 6337
Porcentaje total del dataset: 7.59%
Columnas: 10


Veamos los tipos de datos y qué contienen las variables

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6337 entries, 12 to 83439
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6307 non-null   object 
 1   Gender               6305 non-null   object 
 2   Sexuality            5885 non-null   object 
 3   Country              6337 non-null   object 
 4   EdLevel              6322 non-null   object 
 5   LearnCode            6316 non-null   object 
 6   YearsCode            6254 non-null   object 
 7   YearsCodePro         5160 non-null   object 
 8   Employment           6335 non-null   object 
 9   ConvertedCompYearly  4389 non-null   float64
dtypes: float64(1), object(9)
memory usage: 544.6+ KB


In [10]:
df.describe(include='all')

,Age,Gender,Sexuality,Country,EdLevel,LearnCode,YearsCode,YearsCodePro,Employment,ConvertedCompYearly
count,6307,6305,5885,6337,6322,6316,6254,5160,6335,4.389000e+03
unique,8,11,21,16,9,306,51,43,9,NaN
top,25-34 years old,Man,Straight / Heterosexual,Brazil,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",School,10,3,Employed full-time,NaN
freq,2695,5870,5070,2254,2535,614,514,481,4381,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.906398e+04
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.371509e+05
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.200000e+01
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.567200e+04
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.810500e+04
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.718400e+04


Hay algunas columnas que podrían ser numéricas, pero están como categóricas, ¿por qué?

In [11]:
df.YearsCode.value_counts().sort_values().head()

48                    1
47                    1
More than 50 years    1
50                    1
44                    2
Name: YearsCode, dtype: int64

In [12]:
df.YearsCodePro.value_counts().sort_values().head()

More than 50 years    1
38                    1
43                    1
42                    2
34                    5
Name: YearsCodePro, dtype: int64

Como solo hay 1 valor de "más de 50 años" para cada columna, se lo reemplazará con `50`. Y los valores de "menos de 1 año" con `0.5`. De esa manera se podrá tener columnas numéricas.

In [13]:
df['YearsCode'] = pd.to_numeric(df.YearsCode.replace(['More than 50 years', 'Less than 1 year'], [50, 0.5]))
df['YearsCodePro'] = pd.to_numeric(df.YearsCodePro.replace(['More than 50 years', 'Less than 1 year'], [50, 0.5]))

### Reducir categorías

Hay varias preguntas que tienen múltiples opciones, pero son bajas porcentualmente. Se las reducirá a menos categorías para simplificar el análisis.

#### Género

In [14]:
df.Gender.value_counts(normalize=True)

Man                                                                         0.931007
Woman                                                                       0.039492
Prefer not to say                                                           0.009516
Non-binary, genderqueer, or gender non-conforming                           0.007454
Man;Or, in your own words:                                                  0.004124
Or, in your own words:                                                      0.003331
Man;Non-binary, genderqueer, or gender non-conforming                       0.002855
Woman;Non-binary, genderqueer, or gender non-conforming                     0.001427
Non-binary, genderqueer, or gender non-conforming;Or, in your own words:    0.000317
Man;Woman;Non-binary, genderqueer, or gender non-conforming                 0.000317
Man;Woman                                                                   0.000159
Name: Gender, dtype: float64

In [15]:
df.Gender.where(df.Gender.isin(['Man', 'Woman']), 'Other', inplace=True)
df.Gender.value_counts(normalize=True)

Man      0.926306
Woman    0.039293
Other    0.034401
Name: Gender, dtype: float64

#### Edad

In [16]:
df.Age.value_counts(normalize=True)

25-34 years old       0.427303
18-24 years old       0.241319
35-44 years old       0.212938
45-54 years old       0.070398
Under 18 years old    0.030442
55-64 years old       0.013794
Prefer not to say     0.002537
65 years or older     0.001268
Name: Age, dtype: float64

In [17]:
df.Age.where(~(df.Age.isin(['45-54 years old', '55-64 years old', '65 years or older'])), '> 45 years old', inplace=True)
df.drop(df.Age[df.Age == 'Prefer not to say'].index, inplace=True)
df.Age.value_counts(normalize=True)

25-34 years old       0.428390
18-24 years old       0.241933
35-44 years old       0.213480
> 45 years old        0.085678
Under 18 years old    0.030520
Name: Age, dtype: float64

#### Sexuality

In [18]:
df.Sexuality.value_counts(normalize=True)

Straight / Heterosexual                                   0.862862
Bisexual                                                  0.048211
Prefer not to say                                         0.028450
Gay or Lesbian                                            0.026576
Prefer to self-describe:                                  0.013118
Straight / Heterosexual;Bisexual                          0.007155
Queer                                                     0.003578
Straight / Heterosexual;Prefer to self-describe:          0.002726
Bisexual;Queer                                            0.002215
Bisexual;Gay or Lesbian                                   0.001022
Gay or Lesbian;Queer                                      0.000681
Bisexual;Prefer to self-describe:                         0.000681
Straight / Heterosexual;Queer                             0.000511
Bisexual;Gay or Lesbian;Queer                             0.000511
Straight / Heterosexual;Bisexual;Gay or Lesbian           0.00

In [19]:
df.Sexuality.where(df.Sexuality.isin(['Straight / Heterosexual']), 'LGBT / Non-hetero', inplace=True)
df.Sexuality.value_counts(normalize=True)

Straight / Heterosexual    0.801297
LGBT / Non-hetero          0.198703
Name: Sexuality, dtype: float64

#### Empleo

In [20]:
df.Employment.value_counts(normalize=True)

Employed full-time                                      0.693148
Independent contractor, freelancer, or self-employed    0.121855
Student, full-time                                      0.081025
Employed part-time                                      0.034657
Not employed, but looking for work                      0.033075
Student, part-time                                      0.022788
Not employed, and not looking for work                  0.007913
I prefer not to say                                     0.004115
Retired                                                 0.001424
Name: Employment, dtype: float64

In [21]:
df.Employment.where(~(df.Employment.isin(['Employed full-time', 'Employed part-time'])), 'Employed', inplace=True)
df.Employment.where(~(df.Employment.isin(['Student, full-time', 'Student, part-time'])), 'Student', inplace=True)
df.Employment.where(~(df.Employment.isin(['Not employed, but looking for work', 'Not employed, and not looking for work', 'Retired'])), \
                    'Not employed', inplace=True)
df.drop(df.Employment[df.Employment == 'I prefer not to say'].index, inplace=True)  
df.Employment.value_counts(normalize=True)

Employed                                                0.730812
Independent contractor, freelancer, or self-employed    0.122358
Student                                                 0.104243
Not employed                                            0.042587
Name: Employment, dtype: float64

#### Dónde aprendió a programar

Se cambia las categorías por 3: `Tradicional`, `No tradicional`, `Ambos`

In [22]:
df.LearnCode.value_counts(normalize=True)

School                                                                                                                             0.097355
Other online resources (ex: videos, blogs, etc)                                                                                    0.040153
Books / Physical media                                                                                                             0.035851
School;Books / Physical media                                                                                                      0.031549
Other online resources (ex: videos, blogs, etc);School;Online Courses or Certification;Books / Physical media                      0.027884
                                                                                                                                     ...   
Coding Bootcamp;Other online resources (ex: videos, blogs, etc);School;Other (please specify):;Colleague;Books / Physical media    0.000159
Other online resourc

In [23]:
typeEdu = []
for i in list(df.LearnCode.values):
    if pd.isnull(i):
        typeEdu.append(np.nan)
    elif i == ('School'):
        typeEdu.append('Traditional')
    elif str(i).find('School') == -1:
        typeEdu.append('Non-traditional')
    else:
        typeEdu.append('Both')

df['LearnCode'] = typeEdu

### Agregar data

Columna para conocer cuántos años tomó desde que se aprendió a programar hasta que se lo hizo profesionalmente.

In [24]:
df['YearsLearnPro'] = abs(df.YearsCode - df.YearsCodePro)
df.drop(columns=['YearsCode', 'YearsCodePro'], inplace=True)

Conocer si tiene o no un título universitario.

In [25]:
df.EdLevel.value_counts()

Bachelor’s degree (B.A., B.S., B.Eng., etc.)                                          2529
Some college/university study without earning a degree                                1303
Master’s degree (M.A., M.S., M.Eng., MBA, etc.)                                        910
Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)     529
Professional degree (JD, MD, etc.)                                                     430
Associate degree (A.A., A.S., etc.)                                                    264
Other doctoral degree (Ph.D., Ed.D., etc.)                                             148
Primary/elementary school                                                               92
Something else                                                                          79
Name: EdLevel, dtype: int64

In [26]:
not_degree = ['Some college/university study without earning a degree', 'Primary/elementary school',
              'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)', 'Something else']

df['Degree'] = df.EdLevel.where(df.EdLevel.isin(not_degree), 'Yes')
df['Degree'] = df.Degree.where(df.Degree == 'Yes', 'No')

In [27]:
df.Degree.value_counts(normalize=True)

Yes    0.681811
No     0.318189
Name: Degree, dtype: float64

# Load

El dataset está listo para poder trabajar. Así que se lo exportará a dos lugares distintos:

#### 1. A una base de datos PostgreSQL en la nube:

In [28]:
# Environment variables
HOST = os.environ["HOST"]
DATABASE = os.environ["DATABASE"]
USER = os.environ["USER"]
PORT = int(os.environ["PORT"])
PASSWORD = os.environ["PASSWORD"]

In [32]:
# Create the engine (created with environment variables)
engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

df.to_sql('survey_s_2021', con=engine, index=True, index_label='ResponseId', if_exists='replace')

#### 2. Como archivo csv al directorio `/data/processed/`:

In [31]:
path_processed = os.path.join(os.getcwd(), os.pardir, 'data', 'processed')
df.to_csv(os.path.join(path_processed, 'survey.csv'))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=09491c61-3767-4289-98fd-88aee19bb45d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>